In [0]:
import cv2, os, keras, pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential, Model, load_model, model_from_json
from keras.layers import LSTM, Dense,Embedding, Dropout,Input, GRU, Conv1D, MaxPooling1D, Flatten, TimeDistributed, BatchNormalization, Conv2D, MaxPooling2D, Activation
from keras import regularizers
from keras.optimizers import *
from keras.layers.merge import concatenate

# from keras import backend as K
# K.set_image_data_format('channels_first')

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········


In [0]:
data_file = '/content/drive/My Drive/pickles/test_berkeley_skeleton_images.pickle' # redefined varaible in case you have completed above steps previously.
print('Trying to load pickle from %s' % data_file)
with open(data_file, 'rb') as file:
    svhn_datasets = pickle.load(file)
    train_dataset = svhn_datasets['test_dataset']
print('Pickle Loaded Successfully!')
X = train_dataset['X_test']
X1 = np.asarray(X)
X1 = X1.astype('float32') / np.float32(255)
X1 = np.expand_dims(X1, axis=3)
print(X1.shape)
X2 = []
X3 = []
X4 = []
for i in range (len(X)):
  X2.append(cv2.resize(X[i], None, fx=0.5, fy=0.5, interpolation = cv2.INTER_AREA))
  X3.append(cv2.resize(X[i], None, fx=0.25, fy=0.25, interpolation = cv2.INTER_AREA))
  X4.append(cv2.resize(X[i], None, fx=0.125, fy=0.125, interpolation = cv2.INTER_AREA))

X2 = np.asarray(X2)
X2 = X2.astype('float32') / np.float32(255)
X2 = np.expand_dims(X2, axis=3)
print(X2.shape)
X3 = np.asarray(X3)
X3 = X3.astype('float32') / np.float32(255)
X3 = np.expand_dims(X3, axis=3)
print(X3.shape)
X4 = np.asarray(X4)
X4 = X4.astype('float32') / np.float32(255)
X4 = np.expand_dims(X4, axis=3)
print(X4.shape)

Trying to load pickle from /content/drive/My Drive/pickles/test_berkeley_skeleton_images.pickle
Pickle Loaded Successfully!
(63576, 64, 64, 1)
(63576, 32, 32, 1)
(63576, 16, 16, 1)
(63576, 8, 8, 1)


In [0]:
num_list = [64, 32, 16, 8]
loaded_models = []

# for i in range (len(num_list)):
json_file = open('/content/drive/My Drive/models/encoder_'+str(num_list[0])+'_1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model1 = model_from_json(loaded_model_json)
loaded_model1.load_weights('/content/drive/My Drive/models/encoder_'+str(num_list[0])+'_1.h5')
# loaded_model1.compile(loss = 'binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])
print(loaded_model1.summary())

json_file = open('/content/drive/My Drive/models/encoder_'+str(num_list[1])+'_1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model2 = model_from_json(loaded_model_json)
loaded_model2.load_weights('/content/drive/My Drive/models/encoder_'+str(num_list[1])+'_1.h5')
# loaded_model2.compile(loss = 'binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])
print(loaded_model2.summary())

json_file = open('/content/drive/My Drive/models/encoder_'+str(num_list[2])+'_1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model3 = model_from_json(loaded_model_json)
loaded_model3.load_weights('/content/drive/My Drive/models/encoder_'+str(num_list[2])+'_1.h5')
# loaded_model3.compile(loss = 'binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])
print(loaded_model3.summary())

json_file = open('/content/drive/My Drive/models/encoder_'+str(num_list[3])+'_1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model4 = model_from_json(loaded_model_json)
loaded_model4.load_weights('/content/drive/My Drive/models/encoder_'+str(num_list[3])+'_1.h5')
# loaded_model4.compile(loss = 'binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])
print(loaded_model4.summary())

# loaded_models.append(loaded_model)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 64, 64, 1)         0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 64, 64, 4)         40        
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 32, 32, 4)         0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 32, 32, 8)         296       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 16, 16, 8)         0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 16, 16, 8)         584       
_________________________________________________________________
encoded (MaxPooling2D)       (None, 8, 8, 8)           0         
__________

In [0]:
layer_name = 'encoded'
model1 = Model(inputs= loaded_model1.input, outputs=loaded_model1.get_layer(layer_name).output)

input1 = Input(shape=(num_list[0],num_list[0],1),name = 'image_input1')
modelI1 = model1(input1)
x = Flatten(name = 'flat1')(modelI1)
# # x = Dense(32, activation='elu')
# # x = Dense(11, activation='softmax')(x)
modelI1 = Model(inputs=input1, outputs=x)
modelI1.compile(optimizer='adadelta', metrics=['accuracy'] ,loss='binary_crossentropy')
print(modelI1.summary())


model2 = Model(inputs= loaded_model2.input, outputs=loaded_model2.get_layer(layer_name).output)
input2 = Input(shape=(num_list[1],num_list[1],1),name = 'image_input2')
modelI2 = model1(input2)
x = Flatten(name = 'flat2')(modelI2)
# # x = Dense(32, activation='elu')
# # x = Dense(11, activation='softmax')(x)
modelI2 = Model(inputs=input2, outputs=x)
modelI2.compile(optimizer='adadelta', metrics=['accuracy'] ,loss='binary_crossentropy')
print(modelI2.summary())

model3 = Model(inputs= loaded_model3.input, outputs=loaded_model3.get_layer(layer_name).output)
input3 = Input(shape=(num_list[2],num_list[2],1),name = 'image_input3')
modelI3 = model1(input3)
x = Flatten(name = 'flat3')(modelI3)
# # x = Dense(32, activation='elu')
# # x = Dense(11, activation='softmax')(x)
modelI3 = Model(inputs=input3, outputs=x)
modelI3.compile(optimizer='adadelta', metrics=['accuracy'] ,loss='binary_crossentropy')
print(modelI3.summary())

model4 = Model(inputs= loaded_model4.input, outputs=loaded_model4.get_layer(layer_name).output)
input4 = Input(shape=(num_list[3],num_list[3],1),name = 'image_input4')
modelI4 = model1(input4)
x = Flatten(name = 'flat4')(modelI4)
# # x = Dense(32, activation='elu')
# # x = Dense(11, activation='softmax')(x)
modelI4 = Model(inputs=input4, outputs=x)
modelI4.compile(optimizer='adadelta', metrics=['accuracy'] ,loss='binary_crossentropy')
print(modelI4.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input1 (InputLayer)    (None, 64, 64, 1)         0         
_________________________________________________________________
model_79 (Model)             (None, 8, 8, 8)           920       
_________________________________________________________________
flat1 (Flatten)              (None, 512)               0         
Total params: 920
Trainable params: 920
Non-trainable params: 0
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input2 (InputLayer)    (None, 32, 32, 1)         0         
_________________________________________________________________
model_79 (Model)             multiple                  920       
_________________________________________________________________
flat2 (

In [0]:
y = pd.read_csv('drive/My Drive/y_test.csv', header=None).values
Y = keras.utils.to_categorical(y, num_classes=11)
print(Y.shape)

(63576, 11)


In [0]:
print(X1.shape)

(63576, 64, 64, 1)


In [0]:
layer_name1 = 'flat1'
layer_name2 = 'flat2'
layer_name3 = 'flat3'
layer_name4 = 'flat4'
concat = concatenate([modelI1.get_layer(layer_name1).output, modelI2.get_layer(layer_name2).output, modelI3.get_layer(layer_name3).output, modelI4.get_layer(layer_name4).output], axis=-1)

x = Dense(11, activation='softmax', name='output_layer')(concat)
modelF = Model(inputs=[input1,input2, input3, input4], outputs=x)
# opt = adam(lr = 0.0001)
modelF.compile(optimizer='adadelta', metrics=['accuracy'] ,loss='binary_crossentropy')
print(modelF.summary())

print(len(modelI1.layers))
print(len(modelI2.layers))
print(len(modelI3.layers))
print(len(modelI4.layers))

print(len(modelF.layers))
scores = modelF.evaluate([X1,X2, X3,X4], [Y], batch_size=64, verbose=1)
# print(scores)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_input1 (InputLayer)       (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
image_input2 (InputLayer)       (None, 32, 32, 1)    0                                            
__________________________________________________________________________________________________
image_input3 (InputLayer)       (None, 16, 16, 1)    0                                            
__________________________________________________________________________________________________
image_input4 (InputLayer)       (None, 8, 8, 1)      0                                            
__________________________________________________________________________________________________
model_79 (